 这个是我个人用来train fcn的文件。你们要是有train的好的话可以直接读。

In [1]:
import os
import time
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from keras import backend as K
import tensorflow as tf
from sklearn.model_selection import train_test_split
from google.colab import drive
drive.mount('/content/drive/')

Using TensorFlow backend.


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
#这里按自己的directory设置好了。我是mount在collab上以后做的。
#400张被preprocess好的图。np读进来就好。
data_loading_time_start = time.time()
dir_train_img = '/content/drive/My Drive/ipython notebooks/SemanticSegmentation/2007_training_images.npy'
dir_seg_img = '/content/drive/My Drive/ipython notebooks/SemanticSegmentation/2007_segmented_images.npy'
train_images = np.load(dir_train_img)
seg_images_21 = np.load(dir_seg_img)
print(f'Total time to load data: {time.time() - data_loading_time_start}s')
print(train_images.shape)
print(seg_images_21.shape)

Total time to load data: 5.970752239227295s
(400, 473, 473, 3)
(400, 473, 473, 21)


In [0]:
#这列这个model可以直接load_model读。
n_class = 21
input_img = keras.layers.Input(shape=(473,473,3))

conv1 = keras.layers.Conv2D(filters=128, kernel_size=(5,5), padding='same', data_format='channels_last', strides=3)(input_img)
bn1 = keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv1)
act1 = keras.layers.Activation(activation='relu')(bn1)
pool1 = keras.layers.MaxPool2D((2,2))(act1)

conv2 = keras.layers.Conv2D(filters=256, kernel_size=(4,4), padding='same', data_format='channels_last', strides=2)(pool1)
bn2 = keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv2)
act2 = keras.layers.Activation(activation='relu')(bn2)
pool2 = keras.layers.MaxPool2D((2,2))(act2)

conv3 = keras.layers.Conv2D(filters=512, kernel_size=(3,3), padding='same', data_format='channels_last')(pool2)
bn3 = keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv3)
act3 = keras.layers.Activation(activation='relu')(bn3)
pool3 = keras.layers.MaxPool2D((2,2))(act3)

conv4 = keras.layers.Conv2D(filters=1024, kernel_size=(2,2), padding='same', data_format='channels_last')(pool3)
bn4 = keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv4)
act4 = keras.layers.Activation(activation='relu')(bn4)
pool4 = keras.layers.MaxPool2D((2,2))(act4)

channel_fusion = keras.layers.Conv2D(filters=500, kernel_size=(1,1), padding='same', data_format='channels_last')(pool4)

up1 = keras.layers.UpSampling2D((2,2))(channel_fusion)
conv5 = keras.layers.Conv2D(filters=1024, kernel_size=(4,4), padding='same', data_format='channels_last')(up1)
bn5 = keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv5)
act5 = keras.layers.Activation(activation='relu')(bn5)

up2 = keras.layers.UpSampling2D((3,3))(act5)
conv6 = keras.layers.Conv2D(filters=512, kernel_size=(2,2), padding='same', data_format='channels_last')(up2)
bn6 = keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv6)
act6 = keras.layers.Activation(activation='relu')(bn6)

up3 = keras.layers.UpSampling2D((4,4))(act6)
conv7 = keras.layers.Conv2D(filters=256, kernel_size=(3,3), padding='same', data_format='channels_last')(up3)
bn7 = keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv7)
act7 = keras.layers.Activation(activation='relu')(bn7)

up4 = keras.layers.UpSampling2D((2,2))(act7)
conv8 = keras.layers.Conv2D(filters=128, kernel_size=(4,4), padding='valid', data_format='channels_last')(up4)
bn8 = keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv8)
act8 = keras.layers.Activation(activation='relu')(bn8)

up5 = keras.layers.UpSampling2D((2,2))(act8)
conv9 = keras.layers.Conv2D(filters=n_class, kernel_size=(2,2), padding='valid', data_format='channels_last')(up5)
bn9 = keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv9)
output_img = keras.layers.Activation(activation='softmax')(bn9)

In [9]:
FCNN = keras.models.Model(input_img, output_img)
print(FCNN.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 473, 473, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 158, 158, 128)     9728      
_________________________________________________________________
batch_normalization_1 (Batch (None, 158, 158, 128)     512       
_________________________________________________________________
activation_1 (Activation)    (None, 158, 158, 128)     0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 79, 79, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 40, 40, 256)       524544    
_________________________________________________________________
batch_normalization_2 (Batch (None, 40, 40, 256)       1024

In [10]:
FCNN.compile(
    loss='categorical_crossentropy',
    optimizer = 'adam',
    metrics=['categorical_accuracy'])

In [0]:
X = np.array(train_images)
Y = np.array(seg_images_21)
# x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=42)

In [21]:
FCNN.fit(X, Y, epochs = 10, batch_size = 8, validation_split=.2)

Train on 320 samples, validate on 80 samples
Epoch 1/10
320/320 [==============================] - 50s 157ms/step - loss: 0.4688 - categorical_accuracy: 0.1165 - val_loss: 0.5762 - val_categorical_accuracy: 0.0494
Epoch 2/10
320/320 [==============================] - 38s 120ms/step - loss: 0.4502 - categorical_accuracy: 0.1214 - val_loss: 0.6510 - val_categorical_accuracy: 0.0310
Epoch 3/10
320/320 [==============================] - 38s 119ms/step - loss: 0.4536 - categorical_accuracy: 0.1190 - val_loss: 0.6145 - val_categorical_accuracy: 0.0501
Epoch 4/10
320/320 [==============================] - 38s 119ms/step - loss: 0.4413 - categorical_accuracy: 0.1224 - val_loss: 0.5934 - val_categorical_accuracy: 0.0525
Epoch 5/10
320/320 [==============================] - 38s 119ms/step - loss: 0.4278 - categorical_accuracy: 0.1278 - val_loss: 0.6881 - val_categorical_accuracy: 0.0643
Epoch 6/10
320/320 [==============================] - 38s 118ms/step - loss: 0.4448 - categorical_accuracy: 0.

In [0]:
FCNN.save('/content/drive/My Drive/ipython notebooks/SemanticSegmentation/FCNN_SEAN_.h5')